# Fitting radial velocities

In this tutorial we will learn how to use `jaxoplanet` to compute the radial velocities of a star hosting a single exoplanet, and how to fit this dataset using `numpyro`.

In [ ]:
# Double precision with JAX
import jax

jax.config.update("jax_enable_x64", True)

## Model and dataset

Let's first generate our dataset, consisting in the radial velocities of a star orbited by a unique exoplanet. We start by defining the system (see [Data and Model]() for more details)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from jaxoplanet.orbits import keplerian

star = keplerian.Central(mass=1.0, radius=1.0)
true_planet = keplerian.Body(mass=0.02, period=3.0)
system = keplerian.System(star, bodies=[true_planet])

As we left many parameters as default, let's check the parameters of the system

In [ ]:
system

We can now compute the radial radial_velocities of the star and add some noise to simulate our dataset

In [ ]:
from jaxoplanet.units import unit_registry as ureg

over_time = np.linspace(0, 10, 1000)
over_rvs = system.radial_velocity(over_time)[0]
time = np.sort(np.random.uniform(0, 10, 30))
rv_obs = system.radial_velocity(time)[0]
rv_err = 0.05 * rv_obs.units
rv_obs += rv_err * np.random.normal(size=len(time))

plt.plot(over_time, over_rvs.magnitude)
plt.errorbar(time, rv_obs.magnitude, yerr=rv_err.magnitude, fmt="+k")

## Inference

In [ ]:
import numpyro
import jax.numpy as jnp
from numpyro import distributions as dist, infer


def rv_model(time, params):
    planet = keplerian.Body(mass=params["mass"], period=params["period"])
    system = keplerian.System(central=star, bodies=[planet])
    return system.radial_velocity(time)[0].magnitude


def model(time, y=None):
    mass = numpyro.sample("mass", dist.Uniform(0.01, 0.1))
    period = numpyro.sample("period", dist.Uniform(1.0, 10.0))
    error = numpyro.sample("error", dist.Uniform(0.01, 0.08))
    rv = rv_model(time, {"mass": mass, "period": period})
    numpyro.sample("y", dist.Normal(rv, error), obs=y)

Once the model defined, we will sample the posterior likelihood of the model parameters given the observed radial velocities. As we will need to provide some initial values for these parameters, it is a good idea to check that these values provide a good starting point for the model

In [ ]:
init_values = {"mass": 0.022, "period": 3.01, "error": rv_err.magnitude}
init_model = rv_model(over_time, init_values)

plt.errorbar(time, rv_obs.magnitude, yerr=rv_err.magnitude, fmt="+k")
plt.plot(over_time, init_model, "C0")

In [ ]:
sampler = infer.MCMC(
    infer.NUTS(model, init_strategy=infer.init_to_value(values=init_values)),
    num_warmup=2000,
    num_samples=10000,
    progress_bar=True,
)

sampler.run(jax.random.PRNGKey(6), time, y=rv_obs.magnitude)

In [ ]:
samples = sampler.get_samples()
posterior_rvs = infer.Predictive(model, samples)(jax.random.PRNGKey(0), over_time)["y"]

plt.plot(over_time, posterior_rvs.mean(0), "C0")
plt.fill_between(
    over_time,
    *np.percentile(posterior_rvs, [16, 84], axis=0),
    alpha=0.3,
    color="C0",
)
plt.errorbar(time, rv_obs, yerr=rv_err.magnitude, fmt="+k")